## Introduction:

Main Goals:

1. Identify the recipients that will engage with the campaign.
2. Maximise the campaign’s revenue.


Comments

- The dataset contains only 5% of donors.
- The donations are usually smaller than $20.
- This data is quite noisy, high dimensional.
- There is an inverse relationship between the probability to donate and the amount donated.


Link for dataset and some analysis ==> 

https://github.com/rebordao/kdd98cup

https://github.com/bobbyantonio/KDD98/blob/master/CleanData.py

In [1]:
import tensorflow as tf
import numpy as np
from sklearn import datasets, linear_model
from tensorflow.python.ops import rnn, rnn_cell
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
# from sknn.mlp import Regressor, Layer
import pandas as pd

## RNN
from tensorflow.contrib import rnn 
from keras.models import Sequential
from keras.layers import recurrent
from keras.layers.core import Dense, Dropout, Activation


## plotting .. 
import seaborn as sns
import matplotlib.pyplot as plt
% matplotlib inline

import csv
import sys

import datetime as dt

## warnings
import warnings
warnings.filterwarnings("ignore")

C:\Users\Eman\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
def load_data():
    return pd.read_csv('tuple.csv', header = 0, nrows = 25000)

In [3]:
## I have to run it over all the data not just 100 records
df = load_data()
df.head(3)

,r0,f0,m0,ir0,if0,a,r1,f1,m1,ir1,if1,rew
0,0,0,0,0,0,5,0,1,9.0,0,1,9.0
1,0,0,0,0,0,5,1,0,0.0,0,1,0.0
2,0,0,0,0,0,5,0,1,6.0,0,1,6.0


In [4]:
def df_split():
    
    df = load_data()
    
    train, test = train_test_split(df, test_size = 0.30)  # split data to 50-50 cross validate, Roger 1.6*1000,000
    
    train_x = train[train.columns.difference(['a', 'r0', 'f0', 'm0', 'ir0', 'if0'])]
    train_y = train[train.columns.difference(['rew', 'a', 'r0', 'f0', 'm0', 'ir0', 'if0'])]
    
    test_x = test[test.columns.difference(['a', 'r0', 'f0', 'm0', 'ir0', 'if0'])]
    test_y = test[test.columns.difference(['rew', 'a', 'r0', 'f0', 'm0', 'ir0', 'if0'])]
    
    return train_x, train_y, test_x, test_y

In [5]:
# train_x, train_y, test_x, test_y = df_split()
# np.shape(train_x), np.shape(train_y), np.shape(test_x), np.shape(test_y)

In [6]:
def tuple_(predicted_states):
        
    next_actions = np.zeros([len(predicted_states), 12])
    
    for i in range(11):
        next_actions[:, i] = i

    tuplesMx0 = np.column_stack((predicted_states, next_actions[:,0]))
    tuplesMx1 = np.column_stack((predicted_states, next_actions[:,1]))
    tuplesMx2 = np.column_stack((predicted_states, next_actions[:,2]))
    tuplesMx3 = np.column_stack((predicted_states, next_actions[:,3]))
    tuplesMx4 = np.column_stack((predicted_states, next_actions[:,4]))
    tuplesMx5 = np.column_stack((predicted_states, next_actions[:,5]))
    tuplesMx6 = np.column_stack((predicted_states, next_actions[:,6]))
    tuplesMx7 = np.column_stack((predicted_states, next_actions[:,7]))
    tuplesMx8 = np.column_stack((predicted_states, next_actions[:,8]))
    tuplesMx9 = np.column_stack((predicted_states, next_actions[:,9]))
    tuplesMx10 = np.column_stack((predicted_states, next_actions[:,10]))
    tuplesMx11 = np.column_stack((predicted_states, next_actions[:,11]))
    
    return tuplesMx0, tuplesMx1, tuplesMx2, tuplesMx3, tuplesMx4, tuplesMx5, tuplesMx6, tuplesMx7, tuplesMx8, tuplesMx9, tuplesMx10, tuplesMx11

## Regression phase:

Before performing the prediction task .. let's split the data to training and validation sets .. 

To avoid any problems in prediction by having string variables .. let's binarize (catergorize) all the variables .. 

In [169]:
gamma = 0.9
Num_itrs = 2  # no loop 3leha ( w dah el sa7) .. we just have to run the whole algo. 10 times and report the avg. results

num_epoch = 1 #23 #epochs are cycles of Feedforward and Backprob
batch_size = 100
chunkSize = 100

train_x, train_y, test_x, test_y = df_split()

n_nodes_hl1 = 40 #np.shape(train)[1]
n_nodes_hl2 = 15 #np.shape(train)[0]
# NUM_STATES = np.shape(train)[1]
NUM_DIM =  np.shape(train_x)[1]
num_nodes = np.shape(train_x)[0]
NUM_DIM_output = np.shape(train_y)[1]

num_unrollings = 5

best_actions = np.zeros([np.shape(train_x)[0], batch_size])
Q_optimal = [] 

In [250]:
## for training 
x = tf.placeholder("float", [np.shape(train_x)[0], None, np.shape(train_x)[1]]) 
y = tf.placeholder("float", [np.shape(train_y)[0], np.shape(train_y)[1]])  


## for testing
x_ = tf.placeholder("float", [np.shape(test_x)[0], None, np.shape(test_x)[1]]) 
y_ = tf.placeholder("float", [np.shape(test_y)[0], np.shape(test_y)[1]])  
y

<tf.Tensor 'Placeholder_137:0' shape=(17500, 5) dtype=float32>

In [285]:
# https://github.com/tensorflow/tensorflow/issues/9699
def lstm_model(train_x, train_y, test_x, test_y):
    hidden_dim = 50
    time_step_size = 5
    lstm_size = 480
    rolling = train_x.values.shape[1]
    output_count = 5
    RNN = recurrent.SimpleRNN
    
    with tf.variable_scope("RNN", reuse = True):
    
        train_x = np.reshape(train_x.values, (train_x.shape[0], 1, train_x.shape[1]))
        test_x = np.reshape(test_x.values, (test_x.shape[0], 1, test_x.shape[1]))
        
#         ## model compilation 
        model = Sequential()
        model.add(RNN(hidden_dim,  input_dim = rolling))
        model.add(Dense(output_count))

        model.compile(loss='mean_squared_error', optimizer='rmsprop')
        
        ## model running
        model.fit(train_x, train_y, batch_size = batch_size, 
          nb_epoch = num_epoch, verbose = 0,
        validation_data = (test_x, test_y))
        
        train_predict = model.predict(train_x.astype(int))
        test_predict = model.predict(test_x.astype(int))
        
        return train_predict.astype(int) ## predicted_rewards

In [288]:
train_predict = lstm_model(train_x, train_y, test_x, test_y)
train_predict

array([[0, 0, 0, 0, 0],
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 1],
       ...,
       [0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0],
       [0, 0, 0, 0, 0]])

## Learning the rewards:
Now, the LSTM_RNN output (from validation phase) is considered as input for DQN model, to be able to select the action that has the maximum longtime reward for the customer (highest CLV) .. 

https://arxiv.org/pdf/1602.01580.pdf

## Multilayer Deep Neural Network (DQN):

https://github.com/MorvanZhou/Reinforcement-learning-with-tensorflow/blob/master/contents/5_Deep_Q_Network/RL_brain.py

In [289]:
def DQN_train(x_inputs, y_outputs):

#     print np.shape(y_outputs)
    
    # Placeholder
    X = tf.placeholder(dtype=tf.float32, shape=[None, np.shape(x_inputs)[1] ])
    Y = tf.placeholder(dtype=tf.float32, shape=[None])


    # Model architecture parameters
    n_dim = np.shape(x_inputs)[1] 
    n_neurons_1 = 40
    n_neurons_2 = 15
    n_target = 1 #np.shape(x_inputs)[0]
    
    batch_size = 500 
    epochs = 100
    
    predicted_output = []
    
        # Initializers
    sigma = 1
    
    #First Q Network
    w1 = tf.Variable(tf.random_uniform([n_dim, n_neurons_1], 0, 0.1))
    bias1 = tf.Variable(tf.random_uniform([n_neurons_1], 0, 0.1))
    
    w2 = tf.Variable(tf.random_uniform([n_neurons_1, n_neurons_2], 0, 0.1))
    bias2 = tf.Variable(tf.random_uniform([n_neurons_2], 0, 0.1))
    
    w3 = tf.Variable(tf.random_uniform([n_neurons_2, n_target], 0, 0.1))
    bias3 = tf.Variable(tf.random_uniform([n_target], 0, 0.1))
    
    
    hidden_1 = tf.nn.relu(tf.matmul(X, w1) + bias1)
    hidden_2 = tf.nn.relu(tf.matmul(hidden_1, w2) + bias2)
    y_ = tf.matmul(hidden_2, w3) + bias3
    
    # initialize variables
    init_op = tf.global_variables_initializer()
    
    # Cost function
    mse = tf.reduce_mean(tf.squared_difference(y_, Y))

    # Optimizer
    opt = tf.train.RMSPropOptimizer(0.0001, 0.99).minimize(mse)

    with tf.Session() as sess:
        sess.run(init_op)
        num_itr = int(np.shape(y_outputs)[0] / batch_size)
        
#         for epoch in range(epochs):
#             for i in range(num_itr):
        predicted_output = sess.run(y_, feed_dict={X: x_inputs, Y: y_outputs})
#     print predicted_output
#         print np.shape(predicted_output)       
#         _, c = sess.run([opt, mse], feed_dict={X: x_inputs, Y: y_outputs})
#         print predicted_output
    return predicted_output.astype(int) #[:, 1]

In [290]:
def Q_learning():
    
    avg_Q = []  
    gamma = 0.9
#     actions = actions_prep(df)

# nepisode > https://stats.stackexchange.com/questions/250943/what-is-the-difference-between-episode-and-epoch-in-deep-q-learning
# means one complete path from state, action, next_s, next_a, reward upuntil terminal state

    nepisod = 30 #2 #np.shape(actions)[1]  ##22+ 
    
    predicted_states = lstm_model(train_x, train_y, test_x, test_y)

    
    tuplesMx0, tuplesMx1, tuplesMx2, tuplesMx3, tuplesMx4, tuplesMx5, tuplesMx6, tuplesMx7, tuplesMx8, tuplesMx9, tuplesMx10, tuplesMx11 = tuple_(predicted_states) 
    
    num_rows = np.shape(tuplesMx0)[0] #-1
    
    Q_predicted = np.zeros([num_rows, 12]) #num_rows, num_columns
    
    best_action = np.zeros([num_rows, 1])
#     Q_optimal = np.zeros([nepisod, num_rows])
    
    for i in range(nepisod):
        Q_optimal = train_x['rew'].values.astype(np.float32) + gamma*Q_predicted.max(axis=1) #np.max(Q_predicted) # returns max value per row !
        Q_predicted0 = DQN_train(tuplesMx0, Q_optimal)
        Q_predicted1 = DQN_train(tuplesMx1, Q_optimal)
        Q_predicted2 = DQN_train(tuplesMx2, Q_optimal)
        Q_predicted3 = DQN_train(tuplesMx3, Q_optimal)
        Q_predicted4 = DQN_train(tuplesMx4, Q_optimal)
        Q_predicted5 = DQN_train(tuplesMx5, Q_optimal)
        Q_predicted6 = DQN_train(tuplesMx6, Q_optimal)
        Q_predicted7 = DQN_train(tuplesMx7, Q_optimal)
        Q_predicted8 = DQN_train(tuplesMx8, Q_optimal)
        Q_predicted9 = DQN_train(tuplesMx9, Q_optimal)
        Q_predicted10 = DQN_train(tuplesMx10, Q_optimal)
        Q_predicted11 = DQN_train(tuplesMx11, Q_optimal)
        Q_predicted = np.column_stack((Q_predicted0, Q_predicted1, Q_predicted2, Q_predicted3,
                                          Q_predicted4, Q_predicted5, Q_predicted6, Q_predicted7,
                                          Q_predicted8, Q_predicted9, Q_predicted10, Q_predicted11))

            
    best_action = np.argmax(Q_predicted, axis = 1) 
         
    return best_action, Q_optimal

In [ ]:
best_action, Q_optimal = Q_learning()
best_action, Q_optimal

In [ ]:
np.mean(Q_optimal)

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter('test.xlsx', engine='xlsxwriter')

# Convert the dataframe to an XlsxWriter Excel object.
pd.DataFrame(Q_optimal).to_excel(writer, sheet_name='Sheet1')
pd.DataFrame(best_action).to_excel(writer, sheet_name='Sheet2')

# # Close the Pandas Excel writer and output the Excel file.
writer.save()

## Enhancements:

1. We can use RELU activation function to generate better results in RNN case 

Ref: http://adventuresinmachinelearning.com/recurrent-neural-networks-lstm-tutorial-tensorflow/

2. DDQN